In [64]:
import cv2
import numpy as np
import numpy.random as rd

img = cv2.imread('2_noise.png')
(height,width,channel)=img.shape
print("Image shape is: ", img.shape)

"""
---------- Model ---------------
Use Ising model in the Bishop's Pattern Recognition and Machine Learning book as the image model.
Each pixel= +1 (black) or -1 (white). 
The original image have no preference of black/white, so parameter h is zero.

---------- Energy function ---------------
E(x,y) = - beta * Sum{xi*xj where xi,xj are neighbors} - eta * Sum{xi*yi}
beta, eta are hyperparameters which can be inferred. 
The first term corresponds to correlation between neighboring pixels.
The second term corresponds to the corelation between observed space y with latent space x.

---------- Parameter Prior ---------------
For the parameter prior, we assume that beta and eta are small positive constant with similar scale, 
which corresponds to a comparable neighborhood interaction and latent interaction.
Assume they are all bigger than 1 with a rescaled exponentially decay tails.
Rescale is properly set for hyperparameters to vary.

prior p(beta-1) = c1 * exp( - 4*imgsize * (beta-1))
prior p(eta-1) = c2 * exp( - 4*imgsize * (eta-1))

---------- Full log posterior -------------
p(x,beta,eta|y) = p(x|y,beta,eta) * p(beta) * p(eta)
log[ p(x,beta,eta|y) ] is proportional to: 
( - imgsize * beta) + ( - imgsize * eta) + beta * neighborhoodsum(x) + eta * observedsum(x,y)

------------ Update rule -------------
for xi:
    xi is either 1 or -1 w.p.
        proportional to exp( - [beta*neighbor(xi) + eta*yi] * xi )
    All xi update at the same time.
    
for beta:
    proportional to exp(  [neighborhoodsum(x)-4*imgsize] * beta) 
    The coeficient looks like this because maximum value of neighborhoodsum is 4*imgsize.

for eta:
    proportional to exp(  [observedsum(x,y)-4*imgsize] * beta)
    Similar argument.
"""


# Ising model, vector y is a observable 2D array with element either +1 for black or -1 for white.
y = np.sum(img,axis=2)
y = y/3/255
y = y*2-1
print("Vector shape is: ",y.shape)
# x is the latent variable.
x = np.zeros((y.shape))

imgsize = np.product(y.shape)

def neighborhoodsum(x):
    # return a number which is the sum of all neighborhood pairs
    x1 = np.roll(x,1,axis=0)
    x2 = np.roll(x,-1,axis=0)
    x3 = np.roll(x,1,axis=1)
    x4 = np.roll(x,-1,axis=1)
    ans = np.sum(np.multiply(x,x1)+np.multiply(x,x2)+np.multiply(x,x3)+np.multiply(x,x4))    
    return ans

def observedsum(x,y):
    # return a number which is the sum of all xi*yi
    return np.sum(np.multiply(x,y))

def neighbor(x):
    # return a matrix whose elementwise value is the sum of x's corresponding elementwise neighbors
    (l1,l2)=x.shape
    x1 = np.roll(x,1,axis=0)
    x2 = np.roll(x,-1,axis=0)
    x3 = np.roll(x,1,axis=1)
    x4 = np.roll(x,-1,axis=1)
    return x1+x2+x3+x4

Image shape is:  (342, 481, 3)
Vector shape is:  (342, 481)


In [65]:
steps = 100
# Initialization
rescale = 100000
beta = 1 + rescale*rd.exponential(1/4/imgsize)
eta = 1 + rescale*rd.exponential(1/4/imgsize)
x = y
for m in range(steps):
    # update x
    # c*P(x = 1)
    probmat_positive = np.exp(beta*neighbor(x) + eta*y)
    # c*P(x = -1)
    probmat_negtive = 1/probmat_positive
    # P(x = 1)
    probmat = probmat_positive/(probmat_positive + probmat_negtive)
    x = rd.binomial(1,p = probmat)*2 - 1
    # update beta
    beta = 1 + rescale*rd.exponential(scale = -1/(neighborhoodsum(x)-4*imgsize))
    # update eta
    eta = 1 + rescale*rd.exponential(scale = -1/(observedsum(x,y)-4*imgsize))
print("beta: ",beta)
print("eta: ",eta)

beta:  1.3765592673853704
eta:  1.0000491417068642


In [66]:
x2=(x+1)/2*255
denoised_img = np.stack((x2,x2,x2),axis=2)
cv2.imshow('a',denoised_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [67]:
cv2.imwrite("denoised_2.png",denoised_img)

True